In [6]:
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
import pandas as pd
import numpy as np
import time

max_pid = 42295 # The max number of PIDs on the Worcester Website. See 'link' below. 
pids = np.arange(1,max_pid,1) # Create Array of all the PIDs
cols = ['pid','mblu','year','style','heat','ac','wall','roof','area'] # this is the info to extract 

# function to extract soup object from html document from given url
def get_soup(pid):
    link = "https://gis.vgsi.com/worcesterma/Parcel.aspx?pid=" + str(pid) # main link + pid 
    # Get Html from link
    fp = urllib.request.urlopen(link)
    mybytes = fp.read()
    mystr = mybytes.decode("utf8")
  
    # Create soup object
    soup = BeautifulSoup(mystr, 'html.parser') 
    if soup.find("span",id="MainContent_lblMessage"):
        soup = []
    return soup

# Function to retrieve MBLU from soup
def get_mblu(soup):
    mblue = ""
    mblu = soup.find("span",id="MainContent_lblMblu").text.replace("/  ","-").replace("/","").replace("\\","-")[:-2]
    return mblu 

# Function to retrieve year from soup
def get_year(soup):
    year = 0
    year = soup.find("span",id="MainContent_ctl01_lblYearBuilt").text
    return year 

# Function to retrieve style from soup
def get_style(soup):
    style = ""
    # Find style in html
    style = soup.find("td", text="Style:").find_next_sibling("td").text
    return style

# Function to retrieve style from soup
def get_units(soup):
    units = "1"
    if soup.find("td", text="Occupancy"):
        units = soup.find("td", text="Occupancy").find_next_sibling("td").text
    return units

def get_tot_units(soup):
    tot_units = "1"
    if soup.find("td", text="Residential Units:"):
        tot_units = soup.find("td", text="Residential Units:").find_next_sibling("td").text
    return tot_units

# Function to retrieve heat from soup
def get_heat(soup):
    heat = ""
    if soup.find("td", text="Heat Type:"):
        heat = soup.find("td", text="Heat Type:").find_next_sibling("td").text
    if soup.find("td", text="Heat/AC"):
        heat = soup.find("td", text="Heat/AC").find_next_sibling("td").text
    return heat

# Function to retrieve ac from soup
def get_ac(soup):
    ac = ""
    if soup.find("td", text='AC Type:'):
        ac = soup.find("td", text='AC Type:').find_next_sibling("td").text
    return ac

# Function to retrieve wall type from soup
def get_wall(soup):
    wall = ""
    if soup.find("td", text='Exterior Wall 1'):
        wall = soup.find("td", text='Exterior Wall 1').find_next_sibling("td").text
    if soup.find("td", text='Exterior Wall 1:'):
        wall = soup.find("td", text='Exterior Wall 1:').find_next_sibling("td").text
    return wall

# Function to retrieve roof type from soup
def get_roof(soup):
    roof = ""
    if soup.find("td", text='Roof Cover'):
        roof = soup.find("td", text='Roof Cover').find_next_sibling("td").text
    return roof

# Function to retrieve area from soup
def get_area(soup):
    area = 0
    area = soup.find("span",id="MainContent_ctl01_lblBldArea").text.replace(",","")
    return area 

# Main function to retrieve all the data above from soup and provide DataFrame
def main(pid):
    soup = get_soup(pid)
    if soup:
        mblu = get_mblu(soup)
        year = get_year(soup)
        style = get_style(soup)
        units = get_units(soup)
        tot_units = get_tot_units(soup)
        heat = get_heat(soup)
        ac = get_ac(soup)
        wall = get_wall(soup)
        roof = get_roof(soup)
        area = get_area(soup)
        this_tax = pd.DataFrame([{'pid':pid,'mblu':mblu,'year':year,'style':style,'units':units,'tot_units':tot_units,'heat':heat,'ac':ac,'wall':wall,'roof':roof,'area':area}])
        return this_tax


In [7]:
# Create list
tax_list = []
for ipid in pids:
    # Add each individual tax dataframe to main one
    tax_list.append(main(ipid))#pd.concat([tax,main(ipid)])
# concat list to dataframe
tax=pd.concat(tax_list,ignore_index=True)

In [9]:
# Save DataFrame to CSV
tax.to_csv('scraped_tax_0331.csv', index=False)

In [8]:
tax


,pid,mblu,year,style,units,tot_units,heat,ac,wall,roof,area
0,1,01-009-00001,1915,Two Family,2,1,Steam,None,Aluminum/Vinyl,Asphalt,3160
1,2,01-009-00002,1900,Two Family,2,1,Steam,None,Aluminum/Vinyl,Asphalt,2619
2,3,01-009-00003,1899,Three Fam Conversion,3,1,Steam,None,Aluminum/Vinyl,Asphalt,1560
3,4,01-009-00004,1900,Conventional,1,1,Steam,None,Aluminum/Vinyl,Asphalt,1528
4,5,01-009-00005,,Typical,,1,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...
40540,42290,56-029-36A-R,2005,Half Duplex,1,1,Forced Air,Central,Aluminum/Vinyl,Asphalt,1452
40541,42291,56-029-36B-R,2005,Half Duplex,1,1,Forced Air,Central,Aluminum/Vinyl,Asphalt,1452
40542,42292,56-029-37A-R,2005,Half Duplex,1,1,Forced Air,Central,Aluminum/Vinyl,Asphalt,1452
40543,42293,56-029-37B-R,2005,Half Duplex,1,1,Forced Air,Central,Aluminum/Vinyl,Asphalt,1452
